In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sys
import urllib
from googleapiclient.discovery import build
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [46]:
df = pd.read_csv('../Data/OSG_Podcast_Data2024-02-14.csv')
df['Streams (S)'] = df['Streams (S)'].str.replace(',', '').astype(float)

In [47]:
df['Date Collected'] = pd.to_datetime(df['Date Collected'], format='%m/%d/%y', errors='coerce')
pd.set_option('display.max_colwidth', None)

# Table for Total Plays

In [48]:
most_recent_date = df["Date Collected"].max()
most_recent = df[df["Date Collected"] == most_recent_date]

In [49]:
sorted_df = most_recent.sort_values(by="Total Combined Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Total_plays_df = sorted_df[["Rank", "Episode Title", "Total Combined Plays", "Date"]]
Total_plays_df

,Rank,Episode Title,Total Combined Plays,Date
23,1,David Brooks: What is a Meaningful Life?,22509.0,03-21-2023
34,2,Dr. Tracy Dennis-Tiwary: Can Anxiety Be Good For Us?,17160.0,12-13-2022
11,3,Dr. Rachel Naomi Remen: Can We All Be Healers?,14601.0,09-05-2023
20,4,You’re Not Alone in Feeling Lonely,12485.0,05-02-2023
27,5,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 2),11484.0,02-07-2023
16,6,"Meet My Moai, A Powerful Friendship Tradition",11339.0,06-27-2023
33,7,Bonus: Meditation for Easing Holiday Stress,11019.0,12-20-2022
14,8,Rainn Wilson: Walking A Spiritual Path In A Lonely World,10770.0,07-25-2023
15,9,Richard Reeves: Why Are Boys And Men Struggling For Connection?,10577.0,07-11-2023
18,10,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,10305.0,05-30-2023


# Table for Top 20 youtube plays

In [50]:
sorted_df = most_recent.sort_values(by="Total YouTube Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Top20_YT_plays_df = sorted_df.nlargest(20, "Total YouTube Plays")[["Rank", "Episode Title", "Total YouTube Plays", "Date"]]
Top20_YT_plays_df

,Rank,Episode Title,Total YouTube Plays,Date
34,1,Dr. Tracy Dennis-Tiwary: Can Anxiety Be Good For Us?,15181.0,12-13-2022
33,2,Bonus: Meditation for Easing Holiday Stress,10078.0,12-20-2022
27,3,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 2),9152.0,02-07-2023
23,4,David Brooks: What is a Meaningful Life?,8780.0,03-21-2023
28,5,Bonus: Meditation for Getting Through a Hectic Day,8545.0,01-31-2023
32,6,Bonus: Meditation for Combating Loneliness,7106.0,01-04-2023
39,7,Kal Penn: Taking the Path of Most Resistance,5917.0,10-04-2022
20,8,You’re Not Alone in Feeling Lonely,5758.0,05-02-2023
29,9,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 1),5490.0,01-24-2023
31,10,Maya Shankar: Why is Change So Hard?,5182.0,01-10-2023


# Table for Top 20 Podcast Plays

In [51]:
sorted_df = most_recent.sort_values(by="Total Podcast Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Top20_pod_plays_df = sorted_df.nlargest(20, "Total Podcast Plays")[["Rank", "Episode Title", "Total Podcast Plays", "Date"]]
Top20_pod_plays_df

,Rank,Episode Title,Total Podcast Plays,Date
23,1,David Brooks: What is a Meaningful Life?,13729,03-21-2023
11,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,10355,09-05-2023
14,3,Rainn Wilson: Walking A Spiritual Path In A Lonely World,8921,07-25-2023
12,4,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,8720,08-22-2023
18,5,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,8602,05-30-2023
8,6,Baratunde Thurston: What Can Nature Teach Us About Connection?,8210,10-17-2023
15,7,Richard Reeves: Why Are Boys And Men Struggling For Connection?,8079,07-11-2023
17,8,Judy Woodruff: What Does It Mean To Really Listen?,7964,06-13-2023
16,9,"Meet My Moai, A Powerful Friendship Tradition",7931,06-27-2023
9,10,Ashley Judd & Friends: Finding Chosen Family,7674,10-03-2023


# Table for Apple Engaged Listeners

In [52]:
sorted_df = most_recent.sort_values(by="Unique Engaged Listeners (A)", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Engaged_listens_df = sorted_df[["Rank", "Episode Title", "Unique Engaged Listeners (A)"]]
Engaged_listens_df

,Rank,Episode Title,Unique Engaged Listeners (A)
23,1,David Brooks: What is a Meaningful Life?,2783.0
11,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,1898.0
7,3,Yo-Yo Ma: How Music Heals Us (Part 1),1794.0
22,4,Dr. Rangan Chatterjee: How Stressed Are We? (And What Can We Do About It?) (Part 1),1772.0
14,5,Rainn Wilson: Walking A Spiritual Path In A Lonely World,1629.0
20,6,You’re Not Alone in Feeling Lonely,1608.0
21,7,Dr. Rangan Chatterjee: What's at the Core of Your Happiness? (Part 2),1543.0
12,8,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,1493.0
17,9,Judy Woodruff: What Does It Mean To Really Listen?,1476.0
8,10,Baratunde Thurston: What Can Nature Teach Us About Connection?,1337.0


# Table for Times Played

In [53]:
sorted_df = most_recent.sort_values(by="Times Played", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Times_fully_played_df = sorted_df[["Rank", "Episode Title", "Times Played"]]
Times_fully_played_df.rename(columns={'Times Played': 'Times Fully Played'}, inplace=True)
Times_fully_played_df

,Rank,Episode Title,Times Fully Played
23,1,David Brooks: What is a Meaningful Life?,5003.12
11,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,3004.22
22,3,Dr. Rangan Chatterjee: How Stressed Are We? (And What Can We Do About It?) (Part 1),2979.86
7,4,Yo-Yo Ma: How Music Heals Us (Part 1),2692.54
20,5,You’re Not Alone in Feeling Lonely,2538.95
21,6,Dr. Rangan Chatterjee: What's at the Core of Your Happiness? (Part 2),2536.88
14,7,Rainn Wilson: Walking A Spiritual Path In A Lonely World,2402.13
12,8,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,2267.34
17,9,Judy Woodruff: What Does It Mean To Really Listen?,2144.28
18,10,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,2078.34


# Cumulative Table

In [54]:
def calculate_platform_stats(df, latest):
    # Calculate All Platform Average Plays Per Episode
    all_platform_avg_plays = round(df['Total Combined Plays'].mean())

    # Calculate All Platform Total Plays
    all_platform_total_plays = df['Total Combined Plays'].sum()

    # Calculate All Audio Platform Plays
    all_audio_platform_plays = df['Total Podcast Plays'].sum()

    # Calculate All YouTube Plays
    all_youtube_plays = df['Total YouTube Plays'].sum()
    
    #df['Streams (S)'] = pd.to_numeric(df['Streams (S)'], errors='coerce') # PROBLEM
        
    # Calculate All Spotify Plays
    all_streams = df['Streams (S)'].sum()
    all_hours = df['Hours Played (S)'].sum()
    
    # Calculate All Apple Plays
    apple_engaged_listeners = df['Unique Engaged Listeners (A)'].sum()
    apple_total_time_listened = df['Hours Played (A)'].sum()


    # Extract only the date part from the datetime
    latest_date = latest.date()

    # Create a new DataFrame with the calculated statistics
    stats_df = pd.DataFrame({
    'Metric': ['All Platform Average Plays Per Episode',
               'All Platform Total Plays (Spotify, Apple, YouTube)',
               'All Audio Platform Plays (Spotify + Apple)',
               'All YouTube Plays', 
               'Spotify Listener Streams (60 seconds or more)', 
               'Spotify Consumption Hours',
               'Spotify  Followers',
               'Spotify Ratings',
               'Apple Engaged Listeners (20 minutes or >40%)',
               'Apple Total Time Listened (hours)',
               'Apple  Followers',
               'Apple Ratings'],  # Closing the list of metrics here
        
    latest_date: [int(all_platform_avg_plays), int(all_platform_total_plays),
                  int(all_audio_platform_plays), int(all_youtube_plays), int(all_streams),
                  int(all_hours), df["Followers (S)"].iloc[0], df["Spotify Rating"].iloc[0],  int(apple_engaged_listeners),
                  int(apple_total_time_listened), df["Followers (A)"].iloc[0], df["Apple Rating"].iloc[0]]
    })
    return stats_df



In [55]:
unique_dates = df["Date Collected"].unique()
unique_dates_sorted = sorted(unique_dates, reverse=True)
#latest = df["Date Collected"].max()
latest = unique_dates_sorted[0] if len(unique_dates_sorted) > 1 else None
latest1 = unique_dates_sorted[1] if len(unique_dates_sorted) > 1 else None
latest2 = unique_dates_sorted[2] if len(unique_dates_sorted) > 1 else None
latest3 = unique_dates_sorted[3] if len(unique_dates_sorted) > 1 else None
latest_df = df[df["Date Collected"] == latest]
latest_df1 = df[df["Date Collected"] == latest1]
latest_df2 = df[df["Date Collected"] == latest2]
latest_df3 = df[df["Date Collected"] == latest3]
df1 = calculate_platform_stats(latest_df, latest)
df2 = calculate_platform_stats(latest_df1, latest1)
df3 = calculate_platform_stats(latest_df2, latest2)
df4 = calculate_platform_stats(latest_df3, latest3)
concatenated_df = pd.concat([df4, df3, df2, df1], axis=1)
concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]
concatenated_df

,Metric,2024-01-24,2024-01-29,2024-02-05,2024-02-14
0,All Platform Average Plays Per Episode,7962.0,7941.0,7655.0,8047
1,"All Platform Total Plays (Spotify, Apple, YouTube)",366243.0,373212.0,359786.0,386246
2,All Audio Platform Plays (Spotify + Apple),220255.0,226266.0,229427.0,236099
3,All YouTube Plays,145988.0,146946.0,148297.0,150147
4,Spotify Listener Streams (60 seconds or more),48828.0,49807.0,50896.0,52112
5,Spotify Consumption Hours,17447.0,17925.0,18324.0,18725
6,Spotify Followers,NaN,NaN,NaN,"4,303"
7,Spotify Ratings,NaN,NaN,NaN,4.9 (75)
8,Apple Engaged Listeners (20 minutes or >40%),41645.0,42588.0,43272.0,44688
9,Apple Total Time Listened (hours),38565.0,39651.0,40449.0,41260


# Cumulative Episode

In [56]:
last_eps = most_recent["#"].max() 
last_eps_df = most_recent[most_recent["#"] == last_eps]
last_eps_df

,#,Episode Title,Total Combined Plays,Total Podcast Plays,Total YouTube Plays,Duration,Times Played,Median Play Time (S),% Complete (S),Total Hours Played,...,Unique Engaged Listeners (A),Plays (A),Starts (S),Streams (S),Listeners (S),Date Collected,Followers (A),Followers (S),Spotify Rating,Apple Rating
0,47,Meditation for Connecting with Loved Ones,3737.0,3533,204.0,04:32,1005.88,00:04:31,76,76.0,...,692.0,1486.0,356.0,256.0,295.0,2024-02-14,"6,357","4,303",4.9 (75),4.9 (174)
